In [22]:
import cv2 as cv
import numpy as np
import os
import random

def get_image_names_from_dir(path):
    files = os.listdir(path)
    files = [f"{path}/{file_name}" for file_name in files]
    return files

def gen_kp_ds(kp_list):
    kp_ds = np.zeros((len(kp_list), 4), np.float32)

    for idx in range(len(kp_list)):
        kp = kp_list[idx]
        kp_ds[idx][0] = kp.pt[0]
        kp_ds[idx][1] = kp.pt[1]
        kp_ds[idx][2] = kp.size
        kp_ds[idx][3] = kp.angle
    
    return kp_ds.transpose()

image_names = get_image_names_from_dir("dataset/set1")
image_list = [cv.imread(image) for image in image_names]
image_gray_list = [cv.cvtColor(image, cv.COLOR_BGR2GRAY) for image in image_list]

In [28]:
# Computing keypoints, and descriptors using SIFT algorithm
max_keypoints = 800  # extracts (at max) 800 keypoints
sift = cv.SIFT_create(max_keypoints)
image_kp_des_list = []

for image in image_list:
    kp, des = sift.detectAndCompute(image, None)
    image_kp_des_list.append([kp, des])


image_idx_1 = 10 # 11th image from the set
image_idx_2 = 11 # 12th image from the set

kp_1, d10 = image_kp_des_list[image_idx_1]
kp_2, d11 = image_kp_des_list[image_idx_2]

# extract 200 random keypoint samples
random_sample_1 = random.sample(kp_1, 200)

img_sift_res_1 = cv.drawKeypoints(
    image_gray_list[image_idx_1],
    random_sample_1,
    image_list[image_idx_1],
    flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
)

# cv.imshow("sift res 1", img_sift_res_1)
cv.imwrite("results/set1_200.jpg", img_sift_res_1)

cv.waitKey(0)
cv.destroyAllWindows()

In [29]:
# Matching feature descriptors between two images using Euclidean Distance

bf = cv.BFMatcher()
matches = bf.knnMatch(d10, d11, k=2)
# Apply ratio test to obtain good matches
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:  # You can adjust this threshold
        good_matches.append(m)

# Draw matches
result_img = cv.drawMatches(
    image_list[image_idx_1],
    kp_1,
    image_list[image_idx_2],
    kp_2,
    good_matches,
    None,
    flags=2,
)

# Display the result
cv.imwrite("results/set1_matches.jpg", result_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [14]:
# Applying RANSAC to remove outliers

# Assuming you've obtained 'good_matches' from the previous feature matching step
# Extract keypoints
src_pts = np.float32([kp_1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp_2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Apply RANSAC to find the best homography
homography, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)

# Filter out outliers
inlier_matches = [good_matches[i] for i in range(len(good_matches)) if mask[i] == 1]

# Draw matches with inliers
result_img = cv.drawMatches(
    image_list[image_idx_1],
    kp_1,
    image_list[image_idx_2],
    kp_2,
    inlier_matches,
    None,
    flags=2,
)

# Display the result
cv.imwrite("results/set1_bestMatches.jpg", result_img)
cv.waitKey(0)
cv.destroyAllWindows()